In [2]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse
from typing import List
from sympy import factorint, isprime, gcd
Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return str(p.x)+","+str(p.y) if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")

In [ ]:
from elliptic_curves.l_torsion_group.naive import LTorsionGroup
from support.primes import inverse_primorial, primes_range
from support.quotients import solve_congruence_equations, representative_in_range
from rings.integers.naive import Integers
from rings.polynomials.naive import Polynomials
from rings.quotients.naive import QuotientRing

def frobenius_trace(curve: EllipticCurve):
    trace_congruences = []
    search_range = possible_frobenius_trace_range(curve)
    upper_prime_bound = inverse_primorial(
                            len(search_range),
                            shunned = curve.p
                          )
    
    # Collect the congruence equations (avoid multivariate
    # polynomial arithmetic by handling 2-torsion separately)
    trace_congruences.append( frobenius_trace_mod_2( curve ) )

    torsion_group = LTorsionGroup( curve )
    for prime in primes_range( 3, upper_prime_bound+1 ):
        if prime != curve.p:
            trace_congruences.append(
                    frobenius_trace_mod_l( torsion_group( prime ) )
                 )
    
    # Recover the unique valid trace representative
    trace_congruence = solve_congruence_equations(
                                          trace_congruences
                                      )
    return representative_in_range( trace_congruence, search_range )




def frobenius_trace_mod_2(curve):
    R = Polynomials( curve.p )
    
    x = R(0, 1)
    A, B = curve.a, curve.b

    defining_polynomial = x**3 + A*x + B
    rational_characteristic = x**curve.p - x
    
    # gcd() has an arbitrary unit as leading coefficient;
    # relatively prime polynomials have a constant gcd.
    d = gcd( rational_characteristic, defining_polynomial )
    if d.degree() == 0:
        # The rational characteristic and the defining polynomial
        # are relatively prime: no rational point of order 2 exists
        # and the Frobenius trace must be odd.
        return QuotientRing( Integers, 2 )(1)
    else:
        return QuotientRing( Integers, 2 )(0)




def frobenius_trace_mod_l(torsion_group):
    assert torsion_group.torsion() > 2, \
        "torsion 2 requires multivariate polynomial arithmetic"
        
    ints_mod_torsion = QuotientRing( Integers,
                                     torsion_group.torsion() )
    field_size = torsion_group.curve().field().size()
    
    for trace_candidate in range( 0, torsion_group.torsion() ):
        candidate_congruence = ints_mod_torsion( trace_candidate )
        for point in torsion_group.elements():
            if not frobenius_equation( candidate_congruence,
                                       field_size,
                                       point ):
                # Exit inner loop and skip the 'else' clause.
                break
        else:
            # Execute after the iteration completed; skip upon break.
            return candidate_congruence
    
    message = "Frobenius equation held for no trace candidate"
    raise ArithmeticError( message )


def frobenius_equation(trace, size, point):
    size_remainder = size % trace.modulus()
    result = frobenius( frobenius(point, size), size ) \
                - trace.remainder() * frobenius(point, size) \
                + size_remainder * point
    return result.is_infinite()


def frobenius(point, q):
    return point.__class__( point.x() ** q, point.y() ** q )


def possible_frobenius_trace_range(field):
    return range( -field.p, field.p+1 )

import sys

def naive_schoof_algorithm( p, A, B):
    p, A, B = int(p), int(A), int(B)
    
    order = p + 1 - frobenius_trace(EllipticCurve( p, A, B ))
    print(order)
    return order

# frobenius_trace( EllipticCurve( , A, B ) )
# naive_schoof_algorithm(7919, 1001, 75)
FiniteField(7919)